In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt; plt.rc('font', size=12)
import seaborn as sns
from scipy.stats import entropy
from scipy.stats import percentileofscore
import re

In [19]:
def shift_to_positive(x):
    x = x.copy()
    C = np.absolute(np.nanmin(x)) + 0.01
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if np.isnan(x[i,j]):
                x[i,j] = 0
            else:
                x[i,j] = x[i,j] + C
    return x

def sum_norm(x):
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] = x[i,j]/np.nansum(x)
    return x


def softmax(x):
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if x[i,j] == 0:
                x[i,j] = 0
            else:
                x[i,j] = np.exp(x[i,j])
    x = x / np.sum(x)
    return x

In [20]:
eco_lf0 = pd.read_table('ecological-modeling/mcnulty-results/mcnulty-LF0-seed96/posteriors/interactions/mean_matrix_transf.tsv', index_col=0)
eco_hf0 = pd.read_table('ecological-modeling/mcnulty-results/mcnulty-HF0-seed12/posteriors/interactions/mean_matrix_transf.tsv', index_col=0)

In [21]:
metabo_carbo = pd.read_table('metabolic-modeling/metabolic-metrics/metaddinfo_carbo.tsv', index_col=0)
metabo_amino = pd.read_table('metabolic-modeling/metabolic-metrics/metaddinfo_amino.tsv', index_col=0)
metabo_carbo_amino = pd.read_table('metabolic-modeling/metabolic-metrics/metaddinfo_carbo_amino.tsv', index_col=0)
metabo_int = pd.read_table('metabolic-modeling/metabolic-metrics/metaddinfo_ECs_int.tsv', index_col=0)

In [22]:
# matrix1 = softmax(eco_lf0.values)
matrix1 = shift_to_positive(eco_lf0.values)
# matrix2 = metabo_carbo.values/metabo_carbo.values.sum()
matrix2 = metabo_carbo.values

mean_matrix = (matrix1 + matrix2) / 2
js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
print(js_div)

0.1579356499514533


In [31]:
# matrix1 = softmax(eco_lf0.values)
matrix1 = sum_norm(shift_to_positive(eco_lf0.values))
# matrix2 = metabo_carbo.values/metabo_carbo.values.sum()
matrix2 = sum_norm(metabo_carbo.values)

mean_matrix = (matrix1 + matrix2) / 2
js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
print(js_div)

0.08304812116910543


In [34]:
# js null distribution
js_null = []
for i in range(10000):
    matrix1 = sum_norm(shift_to_positive(eco_lf0.values))
    matrix2_shuffled = np.random.permutation(sum_norm(metabo_carbo.values))
    mean_matrix = (matrix1 + matrix2_shuffled) / 2
    js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2_shuffled.flatten(), mean_matrix.flatten())) / 2
    js_null.append(js_div)
js_null = np.array(js_null)

p_value_ss = percentileofscore(js_null, js_div) / 100
p_value_ss

0.8382000000000001

In [25]:
eco = [eco_lf0, eco_hf0]
eco_name = ['LF0', 'HF0']
metabo = [metabo_carbo, metabo_amino]
metabo_name = ['carbohydrate', 'amino acid']
for i in range(len(eco)):
    for j in range(len(metabo)):
        matrix1 = sum_norm(shift_to_positive(eco[i].values))
        matrix2 = sum_norm(metabo[j].values)
        mean_matrix = (matrix1 + matrix2) / 2
        js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
        print(f'{eco_name[i]} ecological interactions with {metabo_name[j]} metabolism js divergence: {js_div}')

LF0 ecological interactions with carbohydrate metabolism js divergence: 0.08304812116910543
LF0 ecological interactions with amino acid metabolism js divergence: 0.10523692868109667
HF0 ecological interactions with carbohydrate metabolism js divergence: 0.19778242066776275
HF0 ecological interactions with amino acid metabolism js divergence: 0.14681350022249515


In [26]:
for i in range(len(eco)):
    matrix1 = sum_norm(shift_to_positive(eco[i].values))
    matrix2 = sum_norm(metabo_carbo_amino.values)
    mean_matrix = (matrix1 + matrix2) / 2
    js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
    print(f'{eco_name[i]} ecological interactions with carbohydrate and amino acid metabolism js divergence: {js_div}')

LF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: 0.08150406185115816
HF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: 0.16349277144994043


In [27]:
matrix1 = sum_norm(shift_to_positive(eco_lf0.values))
matrix2 = sum_norm(metabo_int.values)
mean_matrix = (matrix1 + matrix2) / 2
js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
print(f'LF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: {js_div}')

LF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: 0.07925671922610927


In [28]:
matrix1 = sum_norm(shift_to_positive(eco_hf0.values))
matrix2 = sum_norm(metabo_int.values)
mean_matrix = (matrix1 + matrix2) / 2
js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
print(f'HF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: {js_div}')

LF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: 0.14729807242092546
